In [6]:
import os
from PIL import Image
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import models, layers
import matplotlib.pyplot as plt
from IPython.display import HTML

aug_dataset_path = r"C:\Users\lenovo\OneDrive\Desktop\potato disease detection\Z_tomato project__02\augmented_dataset"

image_paths = []
for root, dirs, files in os.walk(aug_dataset_path):
    for file in files:
        if file.lower().endswith(('.png', '.jpg', '.jpeg')):
            image_paths.append(os.path.join(root, file))

print(f"Total images found: {len(image_paths)}")


Total images found: 30609


**Train-Test Split**

In [4]:
# !pip install split-folders

Defaulting to user installation because normal site-packages is not writeable


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [7]:
import splitfolders

splitfolders.ratio(aug_dataset_path, output="dataset_split", seed=42, ratio=(.7, .2, .1), group_prefix=None)


Copying files: 30610 files [05:35, 91.28 files/s] 


In [ ]:
# import os
# import random
# import shutil
# from tqdm import tqdm

# def split_dataset(input_dir, output_dir, train_ratio=0.7, val_ratio=0.2, test_ratio=0.1):
#     assert train_ratio + val_ratio + test_ratio == 1.0, "Ratios must sum to 1.0"
    
#     classes = os.listdir(input_dir)
    
#     for cls in tqdm(classes, desc="Processing classes"):
#         cls_path = os.path.join(input_dir, cls)
#         images = os.listdir(cls_path)
#         random.shuffle(images)

#         train_split = int(len(images) * train_ratio)
#         val_split = int(len(images) * (train_ratio + val_ratio))

#         train_imgs = images[:train_split]
#         val_imgs = images[train_split:val_split]
#         test_imgs = images[val_split:]

#         for split_name, split_imgs in zip(['train', 'val', 'test'], [train_imgs, val_imgs, test_imgs]):
#             split_dir = os.path.join(output_dir, split_name, cls)
#             os.makedirs(split_dir, exist_ok=True)
#             for img in split_imgs:
#                 src = os.path.join(cls_path, img)
#                 dst = os.path.join(split_dir, img)
#                 shutil.copyfile(src, dst)

# # Example usage:
# input_dir = 'augmented_dataset'
# output_dir = 'dataset_split'
# split_dataset(input_dir, output_dir)


In [9]:
import os

base_dir = 'aug_dataset_split'

for split in ['train', 'val', 'test']:
    split_path = os.path.join(base_dir, split)
    print(f"\nContents of {split_path}:")
    if os.path.exists(split_path):
        for cls in os.listdir(split_path):
            cls_path = os.path.join(split_path, cls)
            num_files = len(os.listdir(cls_path))
            print(f"  Class '{cls}' has {num_files} images")
    else:
        print(f"{split_path} not found.")



Contents of aug_dataset_split\train:
  Class 'Tomato_Bacterial_spot' has 2214 images
  Class 'Tomato_Early_blight' has 2074 images
  Class 'Tomato_healthy' has 2150 images
  Class 'Tomato_Late_blight' has 2174 images
  Class 'Tomato_Leaf_Mold' has 2082 images
  Class 'Tomato_Septoria_leaf_spot' has 2178 images
  Class 'Tomato_Spider_mites_Two_spotted_spider_mite' has 2172 images
  Class 'Tomato__Target_Spot' has 2144 images
  Class 'Tomato__Tomato_mosaic_virus' has 1988 images
  Class 'Tomato__Tomato_YellowLeaf__Curl_Virus' has 2246 images

Contents of aug_dataset_split\val:
  Class 'Tomato_Bacterial_spot' has 632 images
  Class 'Tomato_Early_blight' has 592 images
  Class 'Tomato_healthy' has 614 images
  Class 'Tomato_Late_blight' has 621 images
  Class 'Tomato_Leaf_Mold' has 595 images
  Class 'Tomato_Septoria_leaf_spot' has 622 images
  Class 'Tomato_Spider_mites_Two_spotted_spider_mite' has 620 images
  Class 'Tomato__Target_Spot' has 612 images
  Class 'Tomato__Tomato_mosaic_vir

**Prepare DataLoaders**

In [12]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_dir = 'aug_dataset_split/train'
val_dir = 'aug_dataset_split/val'
test_dir = 'aug_dataset_split/test'

img_height, img_width = 256, 256  # Or 224, 128, etc.
batch_size = 32

train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen   = ImageDataGenerator(rescale=1./255)
test_datagen  = ImageDataGenerator(rescale=1./255)

train_data = train_datagen.flow_from_directory(train_dir, target_size=(img_height, img_width), batch_size=batch_size, class_mode='categorical')
val_data = val_datagen.flow_from_directory(val_dir, target_size=(img_height, img_width), batch_size=batch_size, class_mode='categorical')
test_data = test_datagen.flow_from_directory(test_dir, target_size=(img_height, img_width), batch_size=batch_size, class_mode='categorical')


Found 21421 images belonging to 10 classes.
Found 6117 images belonging to 10 classes.
Found 3071 images belonging to 10 classes.


**Model Building (CNN Model Architecture)** Example

In [15]:
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)),
    layers.MaxPooling2D(2, 2),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(10, activation='softmax')  # 10 classes
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# model training

history = model.fit(train_data, epochs=5, validation_data=val_data)

Epoch 1/5


C:\Users\lenovo\AppData\Roaming\Python\Python311\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


670/670 ━━━━━━━━━━━━━━━━━━━━ 1130s 2s/step - accuracy: 0.3785 - loss: 1.7932 - val_accuracy: 0.7410 - val_loss: 0.7392
Epoch 2/5
283/670 ━━━━━━━━━━━━━━━━━━━━ 15:47 2s/step - accuracy: 0.7911 - loss: 0.6025

KeyboardInterrupt: 

**Optional: Use EarlyStopping and Checkpoints**

In [ ]:
# from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# callbacks = [
#     EarlyStopping(patience=5, restore_best_weights=True),
#     ModelCheckpoint("best_model.h5", save_best_only=True)
# ]

# history = model.fit(train_data, epochs=50, validation_data=val_data, callbacks=callbacks)


**Evaluate & Save**

In [ ]:
test_loss, test_acc = model.evaluate(test_data)
print(f"Test Accuracy: {test_acc*100:.2f}%")

model.save("tomato_disease_model_v1.h5")

In [ ]:
# !pip install scikit-learn matplotlib seaborn


**Evaluate Model with Metrics**

In [ ]:
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay, accuracy_score, f1_score, precision_score, recall_score
import seaborn as sns
import matplotlib.pyplot as plt

# Get class names
class_names = list(test_data.class_indices.keys())

# Get predictions
y_pred_probs = model.predict(test_data)
y_pred = np.argmax(y_pred_probs, axis=1)

# Get true labels
y_true = test_data.classes

# Accuracy
acc = accuracy_score(y_true, y_pred)
print(f"Accuracy: {acc:.4f}")

# Precision, Recall, F1
precision = precision_score(y_true, y_pred, average='weighted')
recall = recall_score(y_true, y_pred, average='weighted')
f1 = f1_score(y_true, y_pred, average='weighted')

print(f"Precision: {precision:.4f}")
print(f"Recall:    {recall:.4f}")
print(f"F1 Score:  {f1:.4f}")

# Classification Report
print("\nClassification Report:")
print(classification_report(y_true, y_pred, target_names=class_names))

# Confusion Matrix
cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(12, 8))
sns.heatmap(cm, annot=True, fmt='d', xticklabels=class_names, yticklabels=class_names, cmap="Blues")
plt.title("Confusion Matrix")
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.show()


**Training Accuracy & Loss Plots**

In [ ]:
# Plot training history
plt.figure(figsize=(14, 6))

# Accuracy Plot
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Acc')
plt.plot(history.history['val_accuracy'], label='Val Acc')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

# Loss Plot
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()
